In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favorite_color: str

## Write to state

This tool returns a Command that tells LangGraph what to change in the agent state.
It saves the user's favorite color in state, and adds a tool message saying it worked.
The tool_call_id links this message to the exact tool call that triggered it.

In [3]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favorite_color(favorite_color: str, runtime: ToolRuntime) -> Command:
    """
    Update the favorite color of the user in the state once they've revealed it.
    """

    return Command(
        update={
            "favorite_color": favorite_color,
            "messages": [
                ToolMessage(
                    content="Successfully updated favorite color",
                    tool_call_id=runtime.tool_call_id
                )
            ]
        }
    )

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-5-nano",
    tools=[update_favorite_color],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [5]:
from langchain.messages import HumanMessage

user_question = HumanMessage(content="My favorite color is green.")

response = agent.invoke(
    {"messages": [user_question]},
    {"configurable": {"thread_id": "1"}}
)

In [6]:
from pprint import pprint 

pprint(response)

{'favorite_color': 'green',
 'messages': [HumanMessage(content='My favorite color is green.', additional_kwargs={}, response_metadata={}, id='68b7577a-5a42-4374-91ec-7dd122ce4975'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 142, 'total_tokens': 360, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Csd32vjYWylRmSuVpt3cOTLxi0TWg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7181-274b-7aa1-911b-46f74d3fb894-0', tool_calls=[{'name': 'update_favorite_color', 'args': {'favorite_color': 'green'}, 'id': 'call_vXnu600XB9riceHU4fn6lX7X', 'type': 'tool_call'}], usage_metadata=

I can also pass updates to the state myself when I am invoking the agent.

In [7]:
user_question = HumanMessage(content="Hello, how are you?")

response = agent.invoke(
    {
        "messages": [user_question],
        "favorite_color": "blue"
    },
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'favorite_color': 'blue',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='894f8ffc-4bc7-45b3-9df8-119629f3cd65'),
              AIMessage(content='Hello! I’m here and ready to help with whatever you need. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 222, 'prompt_tokens': 142, 'total_tokens': 364, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Csd362RaFxJZciTjuyCPimFUyeEdK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b7181-36c9-7553-96cc-c26e84d92026-0', usage_metadata={'input_tokens': 142, 'output_tokens': 222, 'total_tokens': 364, 'input_tok

In [8]:
agent.get_state(config={"configurable": {"thread_id": "2"}})

StateSnapshot(values={'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='894f8ffc-4bc7-45b3-9df8-119629f3cd65'), AIMessage(content='Hello! I’m here and ready to help with whatever you need. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 222, 'prompt_tokens': 142, 'total_tokens': 364, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Csd362RaFxJZciTjuyCPimFUyeEdK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b7181-36c9-7553-96cc-c26e84d92026-0', usage_metadata={'input_tokens': 142, 'output_tokens': 222, 'total_tokens': 364, 'input_token_details': {'audio

## Read state

In [9]:
@tool
def read_favorite_color(runtime: ToolRuntime) -> str:
    """
    Read the favorite color of the user from the state.
    """

    try:
        return runtime.state["favorite_color"]
    except KeyError:
        return "No favorite color found in state"

agent = create_agent(
    model="gpt-5-nano",
    tools=[update_favorite_color, read_favorite_color],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [10]:
user_question = HumanMessage(content="My favorite color is green.")

response = agent.invoke(
    {"messages": [user_question]},
    {"configurable": {"thread_id": 1}}
)

pprint(response)

{'favorite_color': 'green',
 'messages': [HumanMessage(content='My favorite color is green.', additional_kwargs={}, response_metadata={}, id='b026aef1-f119-47b1-bd26-ca29cc78b51e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 538, 'prompt_tokens': 163, 'total_tokens': 701, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Csd38ZjZ4QKHXLMgl7oqiUKOeltVL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7181-3fbe-7980-af74-3012692893b2-0', tool_calls=[{'name': 'update_favorite_color', 'args': {'favorite_color': 'green'}, 'id': 'call_aYv2zC1CM9VeSlDvT7QvxWD6', 'type': 'tool_call'}], usage_metadata=

https://smith.langchain.com/public/0e04fc13-c561-455b-8058-849fd54f8367/r

In [11]:
user_question = HumanMessage(content="What is my favorite color?")

response = agent.invoke(
    {"messages": [user_question]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favorite_color': 'green',
 'messages': [HumanMessage(content='My favorite color is green.', additional_kwargs={}, response_metadata={}, id='b026aef1-f119-47b1-bd26-ca29cc78b51e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 538, 'prompt_tokens': 163, 'total_tokens': 701, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Csd38ZjZ4QKHXLMgl7oqiUKOeltVL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7181-3fbe-7980-af74-3012692893b2-0', tool_calls=[{'name': 'update_favorite_color', 'args': {'favorite_color': 'green'}, 'id': 'call_aYv2zC1CM9VeSlDvT7QvxWD6', 'type': 'tool_call'}], usage_metadata=

https://smith.langchain.com/public/2ff6f2f5-d883-4b2e-aa29-e2aa77cd4805/r